## Lucid: Workload Estimator

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn import preprocessing, metrics
from interpret.glassbox import ExplainableBoostingRegressor


sns.set_style("ticks")
font = {
    "font.family": "Roboto",
    "font.size": 12,
}
sns.set_style(font)
paper_rc = {
    "lines.linewidth": 3,
    "lines.markersize": 10,
}
sns.set_context("paper", font_scale=1.8, rc=paper_rc)
current_palette = sns.color_palette()

pd.set_option("display.max_columns", None)

idx = 0
save = False
experiment_list = ["Venus_Sept", "Saturn_Sept", "Philly"]
cluster_list = ["Venus", "Saturn", "Philly"]
cluster = cluster_list[idx]
experiment = experiment_list[idx]

datapath = f"../data/{cluster}"


result = pd.DataFrame()
if cluster == "Philly":
    df = pd.read_csv(
        f"{datapath}/cluster_full_log.csv",
        parse_dates=["submit_time"],
        usecols=[
            "job_id",
            "user",
            "vc",
            "gpu_num",
            "submit_time",
            "amp",
            "gpu_util",
            "gmem_util",
            "gmem",
            "duration",
        ],
    )
else:
    df = pd.read_csv(
        f"{datapath}/cluster_full_log.csv",
        parse_dates=["submit_time"],
        usecols=[
            "job_id",
            "user",
            "vc",
            # "jobname",
            "gpu_num",
            "cpu_num",
            "submit_time",
            "month",
            "day",
            "hour",
            "dayofweek",
            "amp",
            "gpu_util",
            "gmem_util",
            "gmem",
            "duration",
        ],
    )

if cluster == "Philly":
    trace_range = ("2017-10-01 00:00:00", "2017-10-07 23:59:00")
    train_df = df[(df["submit_time"] > trace_range[1])]
    val_df = df[(df["submit_time"] >= trace_range[0]) & (df["submit_time"] <= trace_range[1])]
else:
    # trace_range = ("2020-09-01 00:00:00", "2020-09-26 23:59:59")
    trace_range = ("2020-09-01 00:00:00", "2020-09-27 00:10:00")  # Add a bit more job for prediction
    train_df = df[(df["submit_time"] < trace_range[0])]
    val_df = df[(df["submit_time"] >= trace_range[0]) & (df["submit_time"] <= trace_range[1])]


train_df = train_df.sort_values(by="submit_time")
train_df.reset_index(inplace=True, drop=True)
val_df = val_df.sort_values(by="submit_time")
val_df.reset_index(inplace=True, drop=True)

train_data = train_df.drop(columns=["duration", "submit_time"])
test_data = val_df.drop(columns=["duration", "submit_time"])
train_label = train_df[["duration"]]
test_label = val_df[["duration"]]

Weekly Update Lucid Model

In [ ]:
trace_range_list = [
    ("2020-09-01 00:00:00", "2020-09-07 00:00:00"), # Week 1
    ("2020-09-07 00:00:00", "2020-09-14 00:00:00"), # Week 2
    ("2020-09-14 00:00:00", "2020-09-21 00:00:00"), # Week 3
    ("2020-09-21 00:00:00", "2020-09-27 00:10:00"), # Week 4
]
week_df = pd.DataFrame()
for trace_range in trace_range_list:

    train_df = df[(df["submit_time"] < trace_range[0])]
    val_df = df[(df["submit_time"] >= trace_range[0]) & (df["submit_time"] <= trace_range[1])]


    train_df = train_df.sort_values(by="submit_time")
    train_df.reset_index(inplace=True, drop=True)
    val_df = val_df.sort_values(by="submit_time")
    val_df.reset_index(inplace=True, drop=True)

    train_data = train_df.drop(columns=["duration", "submit_time"])
    test_data = val_df.drop(columns=["duration", "submit_time"])
    train_label = train_df[["duration"]]
    test_label = val_df[["duration"]]

    print(f"Train Data Len: {len(train_data)}")

    ebm = ExplainableBoostingRegressor(learning_rate=0.01, interactions=20)
    ebm.fit(train_data, train_label)
    pred = ebm.predict(test_data)

    mae_score = metrics.mean_absolute_error(test_label, pred)
    mape_score = metrics.mean_absolute_percentage_error(test_label, pred)
    r2_score = metrics.r2_score(test_label, pred)
    result.at["ebm_r2", cluster] = r2_score
    print(f"mae_score: {mae_score:.2f}, mape_score: {mape_score:.2f}, r2_score: {r2_score:.4f}")

    pred = pred.astype(int)
    val_df.loc[:,'priority'] = pred
    week_df = pd.concat([week_df, val_df])
# week_df.to_csv(f"ebm/{experiment}_Sept_ebm_weekly_updated.csv", index=False)





Train Data Len: 89556
mae_score: 14435.39, mape_score: 184.21, r2_score: 0.4339
Train Data Len: 94471


KeyboardInterrupt: 

# Job Name Affinity Propagation

Scripts below need original jobname information, which cannot release

In [ ]:
import distance
import time
import warnings
import random
import pandas as pd
import numpy as np
from itertools import groupby
from sklearn.cluster import AffinityPropagation
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore")

top_high_freq_num = 100

idx = 1
experiment_list = ["Venus_Sept", "Saturn_Sept", "Philly"]
cluster_list = ["Venus", "Saturn", "Philly"]
cluster = cluster_list[idx]
experiment = experiment_list[idx]

df = pd.read_csv(f'../data/{cluster}/cluster_full_log.csv',
                 parse_dates=['submit_time', 'start_time', 'end_time'])

df = df[df['gpu_num']>0]
df.drop(columns=['year', 'nodelist', 'priority', 'minute'], inplace=True)
df.reset_index(drop=True, inplace=True)

count = df['jobname'].value_counts()
name_list = list(count.index)

high_freq = name_list[:top_high_freq_num]
to_cluster = name_list[top_high_freq_num:]
to_cluster.sort()

groups = [list(g) for k, g in groupby(to_cluster, key=lambda x: x[0])]

In [ ]:
label_dict = {}

for group in groups:
    if len(group) == 1:
        label_dict.update({group[0]:group[0]})
    else:
        print(f"Processing First Character: {group[0][0]}, Lenth: {len(group)}")
        ts = time.time()

        names = np.asarray(group)
        lev_similarity = -1 * np.array([[distance.levenshtein(w1, w2) for w1 in names] for w2 in names])

        affprop = AffinityPropagation(affinity="precomputed", damping=0.9, random_state=6)
        affprop.fit(lev_similarity)

        for cluster_id in np.unique(affprop.labels_):
            exemplar = names[affprop.cluster_centers_indices_[cluster_id]]
            cluster = np.unique(names[np.nonzero(affprop.labels_==cluster_id)])

            for ori in cluster:
                label_dict.update({ori:exemplar})

        print(f"Time Cost: {time.time()-ts} s")

for i in high_freq:
    label_dict.update({i:i})
assert len(label_dict) == len(name_list)

In [ ]:
"""Replace Name"""
for i in range(len(df)):
    df.at[i, 'jobname'] = label_dict[df.at[i, 'jobname']]

df.to_csv(f"./{cluster}/cluster_full_log.csv", index=None)